In [4]:
from transformers import RobertaConfig, RobertaModel, RobertaModel, RobertaTokenizer
import pandas as pd
import numpy as np
import nltk
from sklearn.metrics.pairwise import cosine_similarity
import torch
from sentence_transformers import SentenceTransformer, util

In [5]:
roberta= SentenceTransformer('sentence-transformers/stsb-roberta-base-v2')

In [6]:
path=r"C:\Users\uttam\OneDrive\Desktop\NLP Project\NLP-Project\NLP-Dataset.csv"
df=pd.read_csv(path,sep=None,engine='python')

In [11]:
train_data=df['sentence'].loc[0:3]
test_data=df['sentence'].loc[3:]
train_data

0    My name is uttam asodariya
1        I am ambitious student
2             I like travelling
3             I cannot here you
Name: sentence, dtype: object

In [12]:
encoded_tensor=roberta.encode(train_data)      
encoded_tensor

array([[-5.3699289e-02, -7.7146357e-01,  4.6951285e-01, ...,
        -1.0547037e+00,  4.5002127e-01, -7.3245448e-01],
       [ 3.9583394e-01, -5.8920127e-01,  2.8462794e-01, ...,
         1.5579911e+00,  5.4898500e-02,  9.7355619e-04],
       [ 1.1584374e+00,  1.1814145e+00, -6.0942006e-01, ...,
         3.3660960e-01,  1.1738288e+00, -1.2833810e-01],
       [-8.0734469e-02,  7.9900771e-01,  2.5858545e-01, ...,
        -1.6918252e-01, -6.0843730e-01, -7.4495894e-01]], dtype=float32)